In [11]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
# from sklearn import tree
from sklearn.tree import plot_tree
from sklearn.svm import SVC
# from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from graphviz import render
import numpy as np
from matplotlib.colors import ListedColormap
le = preprocessing.LabelEncoder()

Конечно, давайте реализуем функцию для построения дерева решений с использованием критерия Джини. Вот полная функция:

Эта функция реализует построение дерева решений с использованием критерия Джини. Вы можете настроить параметры max_depth и min_samples_split при создании объекта DecisionTree, чтобы контролировать глубину дерева и минимальное количество образцов для разделения.

In [12]:
class DecisionTree:
    def __init__(self, max_depth=None, min_samples_split=2):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split

    def fit(self, X, y):
        self.root = self._build_tree(X, y, depth=0)

    def _build_tree(self, X, y, depth):
        n_samples, n_features = X.shape
        unique_classes, class_counts = np.unique(y, return_counts=True)
        dominant_class = unique_classes[np.argmax(class_counts)]

        if depth == self.max_depth or n_samples < self.min_samples_split or len(unique_classes) == 1:
            return LeafNode(dominant_class)

        best_split = self._find_best_split(X, y)
        if best_split is None:
            return LeafNode(dominant_class)

        left_indices, right_indices = best_split
        left_subtree = self._build_tree(X[left_indices], y[left_indices], depth + 1)
        right_subtree = self._build_tree(X[right_indices], y[right_indices], depth + 1)

        return DecisionNode(best_split, left_subtree, right_subtree)

    def _find_best_split(self, X, y):
        n_samples, n_features = X.shape
        if n_samples <= 1:
            return None

        # Вычисление Gini impurity для текущего узла
        _, class_counts = np.unique(y, return_counts=True)
        node_gini = 1.0 - sum((count / n_samples) ** 2 for count in class_counts)

        best_gini = 1.0
        best_split = None

        for feature_index in range(n_features):
            # Сортировка данных по значению признака
            sorted_indices = X[:, feature_index].argsort()
            sorted_y = y[sorted_indices]

            class_counts_left = np.zeros(len(unique_classes))
            class_counts_right = class_counts.copy()

            for i in range(1, n_samples):
                current_class = sorted_y[i - 1]
                class_counts_left[current_class] += 1
                class_counts_right[current_class] -= 1

                gini_left = 1.0 - sum((count / i) ** 2 for count in class_counts_left)
                gini_right = 1.0 - sum((count / (n_samples - i)) ** 2 for count in class_counts_right)

                weighted_gini = (i * gini_left + (n_samples - i) * gini_right) / n_samples

                if sorted_y[i - 1] != sorted_y[i] and weighted_gini < best_gini:
                    best_gini = weighted_gini
                    best_split = (feature_index, X[sorted_indices[i - 1], feature_index])

        return best_split

    def predict(self, X):
        predictions = [self._predict_tree(x, self.root) for x in X]
        return np.array(predictions)

    def _predict_tree(self, x, node):
        if isinstance(node, LeafNode):
            return node.predicted_class
        if x[node.split[0]] <= node.split[1]:
            return self._predict_tree(x, node.left_subtree)
        else:
            return self._predict_tree(x, node.right_subtree)

class LeafNode:
    def __init__(self, predicted_class):
        self.predicted_class = predicted_class

class DecisionNode:
    def __init__(self, split, left_subtree, right_subtree):
        self.split = split
        self.left_subtree = left_subtree
        self.right_subtree = right_subtree


```
Эта функция создает класс GradientBoostingClassifier, который может обучать ансамбль деревьев решений на основе градиентного бустинга для задачи классификации. Вы можете настроить параметры, такие как количество деревьев (n_estimators), скорость обучения (learning_rate), максимальную глубину деревьев (max_depth) и минимальное количество образцов для разделения (min_samples_split) при создании объекта класса.

Чтобы обучить модель, вызовите fit с обучающими данными X и метками y. Чтобы сделать предсказания, используйте метод predict с тестовыми данными X.
```

In [13]:
class GradientBoostingClassifier:
    def __init__(self, n_estimators=100, learning_rate=0.1, max_depth=3, min_samples_split=2):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.estimators = []

    def fit(self, X, y):
        # Инициализация предсказаний нулевым значением
        predictions = np.zeros(len(y))

        for _ in range(self.n_estimators):
            # Вычисление остатков
            residuals = y - predictions

            # Создание нового дерева решений
            tree = DecisionTree(max_depth=self.max_depth, min_samples_split=self.min_samples_split)
            tree.fit(X, residuals)

            # Предсказание с учетом шага обучения
            tree_predictions = tree.predict(X)
            predictions += self.learning_rate * tree_predictions

            # Добавление дерева к ансамблю
            self.estimators.append(tree)

    def predict(self, X):
        # Предсказание ансамбля
        predictions = np.zeros(len(X))
        for tree in self.estimators:
            tree_predictions = tree.predict(X)
            predictions += self.learning_rate * tree_predictions
        return np.round(predictions).astype(int)


#Подготовка данных
удалиv столбцы, которые не будут полезными для классификации микроорганизмов ('Isolate Id', 'Study', 'Year',...)

In [14]:
#загрузка данных из таблиц
data  = pd.read_csv("data.csv",sep=";", encoding="utf-8")

data

/tmp/ipykernel_4892/976355735.py:16: DtypeWarning: Columns (5,19,20,23,24,29,30,35,36,43,44,55,56,57,58,59,61,62,63,64,65,67,68,69,70,71,72,73,75,76,77,78,79,80,81,83,84,85,86,87,88,89,90,91,93,94,95,96,97,98,99,100) have mixed types. Specify dtype option on import or set low_memory=False.
  data  = pd.read_csv("data.csv",sep=";", encoding="utf-8")


,Isolate Id,Study,Species,Organism Group,Country,State,Gender,Age Group,Speciality,Source,...,Sulbactam,Sulbactam_I,Teicoplanin,Teicoplanin_I,Tetracycline,Tetracycline_I,Trimethoprim sulfa,Trimethoprim sulfa_I,Ceftolozane tazobactam,Ceftolozane tazobactam_I
0,1000000,TEST,Pseudomonas aeruginosa,Non-Enterobacteriaceae,France,NaN,Male,85 and Over,Emergency Room,GU: Urine,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1000001,TEST,Pseudomonas aeruginosa,Non-Enterobacteriaceae,France,NaN,Female,13 to 18 Years,Emergency Room,HEENT: Ears,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1000002,TEST,Pseudomonas aeruginosa,Non-Enterobacteriaceae,France,NaN,Female,65 to 84 Years,Nursing Home / Rehab,GU: Urine,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1000003,TEST,Pseudomonas aeruginosa,Non-Enterobacteriaceae,France,NaN,Male,19 to 64 Years,Medicine General,INT: Skin,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1000004,TEST,Serratia marcescens,Enterobacteriaceae,France,NaN,Male,19 to 64 Years,Medicine General,CVS: Blood,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
633815,999995,TEST,Escherichia coli,Enterobacteriaceae,France,NaN,Male,65 to 84 Years,Emergency Room,GU: Urine,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
633816,999996,TEST,Escherichia coli,Enterobacteriaceae,France,NaN,Female,0 to 2 Years,Emergency Room,GU: Urine,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
633817,999997,TEST,Escherichia coli,Enterobacteriaceae,France,NaN,Female,85 and Over,Emergency Room,GU: Urine,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
633818,999998,TEST,Pseudomonas aeruginosa,Non-Enterobacteriaceae,France,NaN,Female,19 to 64 Years,Surgery General,Bodily Fluids: Peritoneal,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
cols_info = data.columns.to_list();
print(cols_info)

['Isolate Id', 'Study', 'Species', 'Organism Group', 'Country', 'State', 'Gender', 'Age Group', 'Speciality', 'Source', 'In / Out Patient', 'Year', 'Phenotype', 'Amikacin', 'Amikacin_I', 'Amoxycillin clavulanate', 'Amoxycillin clavulanate_I', 'Ampicillin', 'Ampicillin_I', 'Azithromycin', 'Azithromycin_I', 'Cefepime', 'Cefepime_I', 'Cefoxitin', 'Cefoxitin_I', 'Ceftazidime', 'Ceftazidime_I', 'Ceftriaxone', 'Ceftriaxone_I', 'Clarithromycin', 'Clarithromycin_I', 'Clindamycin', 'Clindamycin_I', 'Erythromycin', 'Erythromycin_I', 'Imipenem', 'Imipenem_I', 'Levofloxacin', 'Levofloxacin_I', 'Linezolid', 'Linezolid_I', 'Meropenem', 'Meropenem_I', 'Metronidazole', 'Metronidazole_I', 'Minocycline', 'Minocycline_I', 'Penicillin', 'Penicillin_I', 'Piperacillin tazobactam', 'Piperacillin tazobactam_I', 'Tigecycline', 'Tigecycline_I', 'Vancomycin', 'Vancomycin_I', 'Ampicillin sulbactam', 'Ampicillin sulbactam_I', 'Aztreonam', 'Aztreonam_I', 'Aztreonam avibactam', 'Aztreonam avibactam_I', 'Cefixime', '

In [22]:
# Выбор признаков для обучения
selected_features = ['Species', 'Organism Group', 'Country', 'State', 'Gender', 'Age Group', 'Speciality', 'Year', 'Phenotype']
antibiotic_features = [col for col in data.columns if col.endswith('_I')]

selected_features.extend(antibiotic_features)

# Создание датасета с категориальными признаками, преобразованными в числовой вид
transform_data = data[selected_features].copy()

for feature in selected_features:
    transform_data[feature] = le.fit_transform(transform_data[feature].astype(str))

# Создание целевой переменной y
y = transform_data['Phenotype']

# Удаление целевой переменной из датасета
transform_data.drop(columns=['Phenotype'], inplace=True)

X = transform_data

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [25]:
X_train

,Species,Organism Group,Country,State,Gender,Age Group,Speciality,Year,Amikacin_I,Amoxycillin clavulanate_I,...,Gatifloxacin_I,Gentamicin_I,Moxifloxacin_I,Oxacillin_I,Quinupristin dalfopristin_I,Sulbactam_I,Teicoplanin_I,Tetracycline_I,Trimethoprim sulfa_I,Ceftolozane tazobactam_I
361876,129,0,19,45,2,4,3,3,2,2,...,0,3,3,2,3,0,3,3,2,3
217640,240,8,17,45,0,5,3,12,3,3,...,0,3,3,2,3,0,3,3,2,3
598,240,8,28,45,1,2,5,9,3,3,...,0,3,3,2,3,0,3,3,2,3
498160,129,0,23,45,1,4,11,6,2,0,...,0,3,3,2,3,0,3,3,2,3
591114,246,8,68,45,1,2,10,9,3,3,...,0,3,1,0,3,0,2,3,2,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110268,137,4,17,45,1,4,10,11,3,2,...,0,3,3,2,3,0,3,3,2,3
259178,2,7,28,45,1,2,3,13,2,3,...,0,3,3,2,3,0,3,3,2,3
365838,214,7,70,17,0,4,4,3,2,3,...,0,3,3,2,3,0,3,3,2,3
131932,214,7,70,1,1,2,10,0,2,3,...,0,3,3,2,3,0,3,3,2,3


In [26]:
X_test

,Species,Organism Group,Country,State,Gender,Age Group,Speciality,Year,Amikacin_I,Amoxycillin clavulanate_I,...,Gatifloxacin_I,Gentamicin_I,Moxifloxacin_I,Oxacillin_I,Quinupristin dalfopristin_I,Sulbactam_I,Teicoplanin_I,Tetracycline_I,Trimethoprim sulfa_I,Ceftolozane tazobactam_I
372837,2,7,70,38,1,2,5,3,1,3,...,0,3,3,2,3,0,3,3,2,3
315095,120,1,3,45,0,2,10,1,3,3,...,0,3,3,2,3,0,3,3,2,3
529326,137,4,12,45,1,0,9,7,3,2,...,0,3,3,2,3,0,3,3,2,3
414412,232,0,28,45,1,4,3,4,2,1,...,0,3,3,2,3,0,3,3,2,3
584127,129,0,52,45,0,4,4,8,2,2,...,0,3,3,2,3,0,3,3,2,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178688,240,8,70,4,1,4,10,12,3,3,...,0,2,2,0,3,0,2,3,1,3
113358,240,8,18,45,0,4,3,11,3,3,...,0,2,2,1,3,0,2,3,1,3
550325,103,0,70,34,0,4,10,8,2,1,...,0,3,3,2,3,0,3,3,2,3
128724,214,7,51,45,1,5,2,11,2,3,...,0,3,3,2,3,0,3,3,2,3


In [27]:
y_train

361876    2
217640    4
598       3
498160    6
591114    5
         ..
110268    1
259178    5
365838    5
131932    5
121958    3
Name: Phenotype, Length: 507056, dtype: int64

In [28]:
y_test

372837    5
315095    5
529326    0
414412    5
584127    2
         ..
178688    3
113358    4
550325    2
128724    5
459403    5
Name: Phenotype, Length: 126764, dtype: int64

Теперь мы готовы создать и обучить модель градиентного бустинга с использованием ранее определенных функций:

In [29]:
# Создание и обучение модели градиентного бустинга
gb_classifier = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, min_samples_split=2)
gb_classifier.fit(X_train, y_train)

# Предсказание на тестовых данных
y_pred = gb_classifier.predict(X_test)

# Оценка точности модели
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)
print("Точность модели: {:.2f}%".format(accuracy * 100))


InvalidIndexError: (slice(None, None, None), 0)